In [1]:
import tensorflow as tf
# import tensorflow_yolov8 as yolov8
# import tensorflow_object_detection_api as tfod
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
import torch
from ultralytics import YOLO
import cv2

2023-10-26 16:27:52.658543: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-26 16:27:52.988997: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-26 16:27:52.990649: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-26 16:27:54.103242: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Object Detection Model
object_detection_model = YOLO('yolov8x.pt')

# Car Make Recognition Model
car_make_recognition_model = load_model('car_brands_model_v27.h5')

In [3]:
def single_out_car_and_recognize_make(image):

    if object_detection_model is None:
        return None

    detections = object_detection_model(image)

    # Check if detections is empty or None
    if not detections or len(detections) == 0:
        return None

    # Extract the Results object
    results = detections[0]

    # Debugging: print out the attributes of the results object
    print(dir(results))

    # Extract bounding boxes
    bounding_boxes = results.boxes.xyxy  # Assuming the boxes are in xyxy format

    # TODO: Once you identify the right attribute for labels, replace this line
    # For now, I'm setting labels as an empty list for the code to continue
    labels = []

    # Extract bounding boxes for cars
    car_bounding_boxes = [box for i, box in enumerate(bounding_boxes) if i < len(labels) and results.names[labels[i]] == 'car']

    # Check if there are detected cars
    if not car_bounding_boxes:
        return None

    # Crop the car from the image using the first detected car bounding box
    car_bounding_box = car_bounding_boxes[0]
    car_image = image[int(car_bounding_box[1]):int(car_bounding_box[3]), int(car_bounding_box[0]):int(car_bounding_box[2])]

    # Resize the car image to the input size of the car make recognition model
    car_image = tf.image.resize(car_image, (224, 224))

    # Recognize the make of the car
    car_make = car_make_recognition_model.predict(car_image)

    return car_make

# Example usage
from PIL import Image
import numpy as np

image = np.array(Image.open("kok.jpg"))  # You can change the target size to whatever you require

# Single out the car and recognize the make
car_make = single_out_car_and_recognize_make(image)

# Print the car make
if car_make is not None:
    print(car_make)
else:
    print("Failed to load object detection model or detect car in image.")


0: 480x640 1 car, 569.4ms
Speed: 8.9ms preprocess, 569.4ms inference, 11.5ms postprocess per image at shape (1, 3, 480, 640)


['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply', '_keys', 'boxes', 'cpu', 'cuda', 'keypoints', 'masks', 'names', 'new', 'numpy', 'orig_img', 'orig_shape', 'path', 'plot', 'probs', 'save_crop', 'save_dir', 'save_txt', 'speed', 'to', 'tojson', 'update', 'verbose']
Failed to load object detection model or detect car in image.
